In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
import os
rc('text', usetex=True)
rc('font', family='serif')
import corr_function
from getpath import getpath 

In [2]:
Dmin = -2  
Dmax = 2. 
NBIN = 40
Bw = (Dmax - Dmin)/NBIN
bins = np.array([Dmin + Bw*i for i in range(NBIN)]) + Bw*0.5

In [ ]:
%%time
# ======================================== MOCK CATALOGUES ==================================================
models = ['G13', 'GP14', 'L16']
type_cut = ['Stellar Mass/', 'SFR/']
dc = [4]

for model in models:
    model = model + '_millennium'
    path = "/home/esteban/Escritorio/Practica/Data/" + model + '/Mock Catalogues/'
    for cut in type_cut:
        if cut == type_cut[0]: tc = 'Mstell'
        else: tc = 'SFR'    
        for c in dc:
            print "entre a 1"
            data = np.load(path + cut + 'cat_d%i_2HOD.npy' %c)
            x = data[:,0].astype('float64')
            y = data[:,1].astype('float64')
            z = data[:,2].astype('float64')
            n_cen =  data[:,5].astype('int32')
            n_sat =  data[:,6].astype('int32')
            
            mask = (n_cen == 0) & (n_sat == 0)
            x = x[~mask]
            y = y[~mask]
            z = z[~mask]
            n_cen = n_cen[~mask]
            n_sat = n_sat[~mask]
    
            
            #Len_Cat = np.sum(n_cen) + np.sum(n_sat)
            
            for i in range(len(x)):
                            
                Nsats = n_sat[i]
                if Nsats == 0: continue
                
                Ncent = n_cen[i]
                if Ncent == 0: Ngal = Nsats - 1
                else: Ngal = Nsats
                        
                x = np.append(x, x[i]*np.ones(Ngal))
                y = np.append(y, y[i]*np.ones(Ngal))
                z = np.append(z, z[i]*np.ones(Ngal))                   
                    
            print "todo ok en", model, "para", cut, "y con density",c         
            DD, RR = corr_function.xi_ll(x, y, z, tc, c, path)      
            xi_mock_path = "/home/esteban/Escritorio/Practica/Data/" + model + '/xi_mocks/' + cut
            xi = (DD/RR) -1
            np.savetxt(xi_mock_path + 'xi_cat_0%i_2HOD.txt' %c, np.array([bins, xi, DD, RR]).T)


In [ ]:
%%time
DD, RR = corr_function.xi_ll(x, y, z, 'Mstell', 4, path)

In [ ]:
xi_mock_path = "/home/esteban/Escritorio/Practica/Data/" + model + '/xi_mocks/' + 'Stellar Mass/'
xi = (DD/RR) -1
np.savetxt(xi_mock_path + 'xi_cat_02.txt', np.array([bins, xi, DD, RR]).T)

In [ ]:
xi_mock_path = "/home/esteban/Escritorio/Practica/Data/GP14_millennium/xi_mocks/Stellar Mass"
xi_sam = "/home/esteban/Escritorio/Practica/Data/GP14_millennium/xi_sam/Stellar Mass"
xi_plot = "/home/esteban/Escritorio/Practica/Plots/GP14_millennium"

data_sam = np.loadtxt(xi_sam + '/xi_d3_fixed.txt')
data_mock = np.loadtxt(xi_mock_path + '/xi_cat_03.txt')

bins_sam, logxi_sam = data_sam[:,0], np.log10(data_sam[:,1])
bins_mock, logxi_mock = data_mock[:,0], np.log10(data_mock[:,1])

f = plt.figure()
plt.plot(bins_sam, logxi_sam, 'k-', label = r"$\xi$ from SAM")
plt.plot(bins_mock, logxi_mock, 'k--', label = r"$\xi$ from Mock")
plt.xlabel("$log(d/h^{-1}Mpc)$")
plt.ylabel(r"$log(\xi)$")
plt.ylim(-2,3)
plt.xlim(-2,2)
plt.legend(loc='upper left', frameon=False)
f.savefig(xi_plot + '/xi_cat_d3_Mstell.pdf')

In [3]:
%%time
# ======================= CORRELATION FUNCTION FOR SHUFFLED CATALOGUE =====================================
models = ['G13']

for model in models:
    
    model = model + '_millennium'
    path = "/home/esteban/Escritorio/Practica/Data/" + model
    
    data = np.load(path + '/r.npy')
    x = (data[:,0]).copy(order='C')
    y = (data[:,1]).copy(order='C')
    z = (data[:,2]).copy(order='C')
    del data
    
    DD, RR = corr_function.xi_ll(x, y, z, ['Mstell','SFR'], 4, path, Shuffle=True)
    
    for i, type_cut in enumerate(['Stellar Mass/', 'SFR/']):
        
        path_xi_sam_shuffle = path + '/xi_sam/' + '/Shuffle/' + type_cut
        
        for j, den_cut in enumerate([4]):
            
            xi = DD[i][j]/RR[i][j] - 1
            DD_save = DD[i][j]
            RR_save = RR[i][j]            
            np.savetxt(path_xi_sam_shuffle + 'xi_d%i.txt' %den_cut, np.array([bins, xi, DD_save, RR_save]).T)
            print "Archivo generado: Density cut:",den_cut, type_cut[:-1],'\n'
    print "Termine", model

Archivo generado: Density cut: 4 Stellar Mass 

Archivo generado: Density cut: 4 SFR 

Termine G13_millennium
CPU times: user 1h 48min 10s, sys: 6.04 s, total: 1h 48min 16s
Wall time: 1h 48min 33s


In [ ]:
model = 'G13'
model = model + '_millennium'
path = "/home/esteban/Escritorio/Practica/Data/" + model 

path_xi_sam = path + '/xi_sam/' + 'SFR/'
old_data = np.loadtxt(path_xi_sam + 'xi_2.5.txt')
bins1, logxi1 = old_data[:,0], np.log10(old_data[:,1])

new_data = np.loadtxt(path_xi_sam + 'xi_d3_fixed.txt')
bins2, logxi2 = new_data[:,0], np.log10(new_data[:,1])

plt.plot(bins1,logxi1,'r-')
plt.plot(bins2,logxi2,'b-')
plt.ylabel(r"$log(\xi)$")
plt.xlabel("$log(d/h^{-1}Mpc)$")
plt.title(r"$n = 10^{-2.5} /h^{-3} Mpc^3$")
plt.ylim(-2.5)
plt.xlim(-2,2)

plt.show()

In [ ]:
#model = str(raw_input("Model: ")) + "_millennium"
model = 'G13_millennium'
path = "/home/esteban/Escritorio/Practica/Data/" + model
x = np.load(path + '/x.npy')
y = np.load(path + '/y.npy')
z = np.load(path + '/z.npy')

In [ ]:
# Quiero comprobar todas mis funciones de correlacion!

In [ ]:
DD, RR = corr_function.xi_ll(x,y,z,'Mstell', [1,2], path)


In [ ]:
xi = DD/RR - 1
logxi = np.log10(xi)

In [ ]:
model = str(raw_input("Model: ")) + "_millennium"
cut_type = int(raw_input("Type Cut:\n Stellar Mass: 1\n SFR: 2\n Choice: "))
path = "/home/esteban/Escritorio/Practica/Data/" + model
os.chdir( path )

In [ ]:
x = np.load('x.npy')
y = np.load('y.npy')
z = np.load('z.npy')
sfr = np.load('sfr.npy')
stellarmass = np.load('stellarmass.npy')
print len(x)

# === Density selection ===
#   Fila    log(rho)
#    20      -2.0
#    15      -2.5
#    10      -3.0
#     5      -3.5
# =================
log_rho, mass_cuts, sfr_cuts = np.loadtxt('density_data.txt', unpack = True)
rho, mass_cut, sfr_cut = log_rho[10], mass_cuts[10], sfr_cuts[10]

if cut_type == 1: feature = stellarmass; cut = mass_cut
else: feature = sfr; cut = sfr_cut

mask = feature > cut
x = x[mask]
y = y[mask]
z = z[mask]

In [ ]:
# ===================== RATIO PLOT =====================
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
import os
% matplotlib inline
rc('text', usetex=True)
rc('font', family='serif')

f = plt.figure()
#models = ['G13']
models   = ['G13', 'GP14']
type_cut = ['Stellar Mass/']                    # Modificar
#marker   = ['.', '+', '^']
color    = ['r.', 'b+', 'g^']

pos_rho = [15, 10, 5]
density_cut = [3,2,1]

arr_G13 = np.arange(1,4)[::-1]
arr_G14 = np.arange(1,4)[::-1]
arr_L16 = np.arange(1,4)[::-1]

arr_models = [arr_G13, arr_G14, arr_L16]

i = 0
for ab, arr in zip(models, arr_models):
    
    model = ab + '_millennium'
    path = "/home/esteban/Escritorio/Practica/Data/" + model

    x = np.load(path + '/x.npy')
    sfr = np.load(path + '/sfr.npy')
    stellarmass = np.load(path + '/stellarmass.npy')
    if ab == 'G13': halomass = np.log10(np.load(path + '/centralmvir.npy') + 1e-10) + 10
    else: halomass = np.log10(np.load(path + '/mdhalo.npy') + 1e-10) + 10    
    
    lmin = 10
    lmax = 14.8333
    

    log_rho, mass_cuts, sfr_cuts = np.loadtxt(path + '/density_data.txt', unpack = True)
    
    for prop in type_cut:
        
        cat_path = path + '/Catalogues/' + prop
        ratio_list = []

        for pos, d in zip(pos_rho, density_cut):
             
            rho, mass_cut, sfr_cut = log_rho[pos], mass_cuts[pos], sfr_cuts[pos]

            if prop == 'Stellar Mass/': feature = stellarmass; cut = mass_cut
            else: feature = sfr; cut = sfr_cut

            mask = feature > cut
            x_sam = x[mask]


            data = np.load( cat_path + 'cat_d%i_fixed.npy' %(d) )
            
            ratio_list.append( len(data)/ float(len(x_sam)) )
            
        #ratios = np.array(ratio_list)
    arr = (np.concatenate([arr, ratio_list], axis = 0).reshape(2,3)).T
    plt.plot(arr[:,0], arr[:,1], color[i], label = '%s' %ab)
    i += 1


plt.xlabel('Density Cut')
plt.ylabel('Catalogues Ratio')
plt.ylim(1.188, 1.225)
plt.legend(frameon = True, loc = 'best', numpoints=1)
plt.title('Stellar Mass samples')                    # Modificar
plt.xlim(0,5)
path =  "/home/esteban/Escritorio/Practica/Plots/"
os.chdir( path )
f.savefig("Ratio_Mstell.pdf")                           # Modificar
plt.show()   

In [ ]:
%%time
# ================== FUNCION DE CORRELACIÓN PARA TODAS LAS GALAXIAS ==================================
import numpy as np
import os
models = ['G13', 'GP14', 'L16']
NBIN = 40
Dmin = -2  # Min separation (log(d)) [d] = Mpc
Dmax = 2.  # Max separation (log(d))
bin_width = (Dmax - Dmin)/NBIN
DD_array = np.zeros(NBIN).astype('int32')
bins = np.array([Dmin + bin_width*i for i in range(NBIN)]) + bin_width *.5

def RR (r,n,dr):
    V_c = (4./3)*np.pi*((10**(r+dr))**3 - (10**r)**3)
    V_t = 500.**3
    return (V_c/V_t)*n**2

for model in models:
    
    model = model + "_millennium"
    path = "/home/esteban/Escritorio/Practica/Data/" + model 
    os.chdir( path )
    x = np.load('x.npy')
    y = np.load('y.npy')
    z = np.load('z.npy')
    sfr = np.load('sfr.npy')
    stellarmass = np.load('stellarmass.npy')
    
    log_rho, mass_cuts, sfr_cuts = np.loadtxt('density_data.txt', unpack = True)
    list_mass_cut = [mass_cuts[5], mass_cuts[10], mass_cuts[15]]
    list_sfr_cut = [sfr_cuts[5], sfr_cuts[10], sfr_cuts[15]]
    criteria = [list_mass_cut, list_sfr_cut]
    rhos = [-3.5, -3.0, -2.5]
    
    for cuts in criteria:
        
        if cuts == criteria[0]:  tc = 'Stellar Mass/'
        else: tc = 'SFR/'    
        
        for rho, cut in zip(rhos, cuts):
            
            if cuts == criteria[0]: mask = stellarmass > cut 
            else: mask = sfr > cut
                
            x_temp = x[mask]
            y_temp = y[mask]
            z_temp = z[mask]
     
            path = "/home/esteban/Escritorio/Practica/Codes/C_Python"
            os.chdir( path )
            import SPack
            
            len_data = len(x_temp)
            DD_array = np.zeros(NBIN).astype('int32')    
            
            SPack.xi(DD_array, x_temp, y_temp, z_temp, len_data, Dmin, Dmax, NBIN)
            RR_array = np.array([RR(r, len_data, bin_width) for r in bins])
            
            #logxi = np.log10((2*DD_array/RR_array)-1)
            xi = 2*DD_array/RR_array - 1
            
            path = "/home/esteban/Escritorio/Practica/Data/" + model 
            xi_sam_path = path + '/xi_sam/' + tc
            os.chdir( xi_sam_path )
            
            #if cuts == criteria[0]: j = i+1
            #else:  j = i+5    
            np.savetxt("xi_%.1f.txt" %rho, np.array([bins, xi, DD_array, RR_array]).T)

In [ ]:
%%time
# =================== CORRELATION FUCTION OF MY MOCKS CATALOGUES ===========
import numpy as np
import os
rhos = [3.5, 3.0, 2.5]
cuts   = [1, 2, 3]
models = ['G13', 'GP14']
type_cut = ['Stellar Mass/']

NBIN = 40
Dmin = -2  # Min separation (log(d)) [d] = Mpc
Dmax = 2.  # Max separation (log(d))
bin_width = (Dmax - Dmin)/NBIN
DD_array = np.zeros(NBIN).astype('int32')
bins = np.array([Dmin + bin_width*i for i in range(NBIN)]) + bin_width *.5

def RR (r,n,dr):
    V_c = (4./3)*np.pi*((10**(r+dr))**3 - (10**r)**3)
    V_t = 500.**3
    return (V_c/V_t)*n**2

for model in models:

    #ab = model
    model = model + "_millennium"
    path = "/home/esteban/Escritorio/Practica/Data/" + model
    
    for cut in type_cut:
        
        cat_path = path + '/Catalogues/' + cut
        
        for i in range(3):
            
            data = np.load(cat_path + 'cat_d%i_fixed.npy' %(i+1))
            x, y, z = data[:,0], data[:,1], data[:,2]
            x = x.copy(order='C')
            y = y.copy(order='C')
            z = z.copy(order='C')
            
            path1 = "/home/esteban/Escritorio/Practica/Codes/C_Python"
            os.chdir( path1 )
    
            len_data = len(x)
            DD_array = np.zeros(NBIN).astype('int32')    
            
            import SPack
            SPack.xi(DD_array, x, y, z, len_data, Dmin, Dmax, NBIN)
            
            RR_array = np.array([RR(r, len_data, bin_width) for r in bins])
            xi = (2*DD_array/RR_array) - 1
            
            xi_path = path + '/xi_cat/' + cut
            os.chdir( xi_path )
            
            np.savetxt('xi_cat_0%i.txt' %(i+1), np.array([bins, xi, DD_array, RR_array]).T)

In [ ]:
def RR (r,n,dr):
    V_c = (4./3)*np.pi*((10**(r+dr))**3 - (10**r)**3)
    V_t = 500.**3
    return (V_c/V_t)*n**2

In [ ]:
%%time
#path = "/home/esteban/Escritorio/Practica/Codes/C_Python"
#os.chdir( path )
import SPack as SP

NBIN = 40
Dmin = -2  # Min separation (log(d)) [d] = Mpc
Dmax = 2.  # Max separation (log(d))
bin_width = (Dmax - Dmin)/NBIN

len_data = len(x)
DD_array = np.zeros(NBIN).astype('int32')

# ===================== C - MODULE =========================
#SPack.xi3(DD_array, x, y, z, len_data, Dmin, Dmax, NBIN, 50, 6, 2, 2)
SP.xi2(DD_array, x, y, z, len_data, Dmin, Dmax, NBIN, 50, 6)
# ==========================================================

#bins = np.linspace(Dmin, Dmax, NBIN, endpoint = False)   
bins = np.array([Dmin + bin_width*i for i in range(NBIN)])
bins_p = bins + bin_width * 0.5
RR_array = np.array([RR(r, len_data, bin_width) for r in bins])

In [ ]:
xi = (2*DD_array/RR_array)-1

In [ ]:
logxi = np.log10((2*DD_array/RR_array)-1)

In [ ]:
f = plt.figure()
#path = "/home/esteban/Escritorio/Practica/Plots/" + model
#os.chdir( path )

#plt.title(r"$n = 10^{%.1f} /h^{-3} Mpc^3$" %rho)
plt.plot(bins,logxi,'k.')
plt.ylim(-2,6)
plt.xlim(-2,1.5)
plt.xlabel("$log(d/h^{-1}Mpc)$")
plt.ylabel(r"$log(\xi)$")
#plt.legend(frameon=False)
plt.minorticks_on()
#f.savefig('xi_03.pdf')
plt.show()

In [ ]:
np.save('xi_data_d3', np.array(bins_p, logxi, DD, RR))

In [ ]:
path = "/home/esteban/Escritorio/Practica/Data/G13_millennium/sergio" # Guo2013 
#path = "/home/esteban/Escritorio/Practica/Data/G14_millennium" # Gonzalez2014 
#path = "/home/esteban/Escritorio/Practica/Data/L16_millennium" # Lacey2016 
os.chdir( path )
np.savetxt("xi_data_03.txt", np.array([bins, logxi]).T)

In [ ]:
10**(1.8)

In [ ]:
bins

In [ ]:
DD_array/2

In [ ]:
DD_array/2

In [ ]:
DD_array/2

In [ ]:
DD_array/2

In [ ]:
DD/2